In [1]:
import pandas as pd
import numpy as np
health = pd.read_csv('data/cleaned_trainingdata.csv')
health = health.set_index('person_id_syn')

In [2]:
health.head()

,betos_d1c_ind,betos_d1d_ind,betos_m1b_ind,betos_m2c_ind,betos_m5b_ind,betos_m5c_ind,betos_m5d_ind,betos_o1a_ind,betos_o1b_ind,betos_o1e_ind,...,rx_common_total,rx_common_total_pct,rx_tired_total,rx_tired_total_pct,rx_severe_total,rx_severe_total_pct,rx_motorability_total,rx_motorability_total_pct,RxMobileIndex,RxMobileIndex_pct
person_id_syn,,,,,,,,,,,,,,,,,,,,,
0002MOb79ST17bLYAe46eIc2,0,0,1,0,0,0,0,0,0,0,...,0.193999,0.061958,0.193999,0.061958,0.193999,0.061958,0.193999,0.061958,0.000689,0.061958
0004cMOS6bTLf34Y7AIca8f3,0,0,1,0,0,1,1,0,0,0,...,1.744756,0.140484,1.744756,0.140484,1.744756,0.140484,1.744756,0.140484,0.060844,0.140484
000536M9O3ST98LaYaeA29Ia,0,0,1,0,0,1,1,1,0,1,...,4.913333,0.400511,4.913333,0.400511,4.913333,0.400511,4.913333,0.400511,0.170698,0.400511
0009bMO9SfTLYe77A51I4ac3,0,0,1,0,0,0,1,0,0,0,...,1.250000,0.129915,1.250000,0.129915,1.250000,0.129915,1.250000,0.129915,0.039753,0.129915
000M7OeS66bTL8bY89Aa16Ie,0,0,1,0,0,0,0,0,0,1,...,0.833333,0.073710,0.833333,0.073710,0.833333,0.073710,0.833333,0.073710,0.028193,0.073710


In [3]:
Y = health['transportation_issues']
X = health.drop('transportation_issues', axis = 1)

In [4]:
from sklearn.model_selection import train_test_split

X_train82, X_holdout_test, y_train82, y_holdout_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train82, y_train82, test_size=0.2, random_state=42)

In [5]:
def ForwardSelection(train_features, train_labels, test_features, test_labels):
    '''
    Completes a forward selection iterative model, starting with no features and adding a new feature
    based on the best ROC AUC score. This program, for each iteration, will print the best feature that was 
    added to the model and it's associated ROC AUC score.
    
    Takes 4 arguments: 
    
    train_features: training dataset with no labels
    train_labels: training dataset with only target labels
    test_features: test dataset with no labels
    test_labels: test dataset with only target variables
    '''
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score
    
    best_roc = 0
    best_roc_list = [0]
    checks_before_break = 0
    features = list(train_features.columns)
    best_features = []

    while (len(features) >= 2): 
        for feature in features: 
            if best_features == []:
                model = LogisticRegression(solver='liblinear', random_state=0)
                model.fit(np.array(train_features[feature]).reshape(-1,1), train_labels)
                p_pred = model.predict_proba(np.array(test_features[feature]).reshape(-1,1))
                p_pred = p_pred[:,1]
                roc_value = roc_auc_score(test_labels, p_pred)
            else:
                features_to_test = best_features + [feature]
                model = LogisticRegression(solver='liblinear', random_state=0)
                model.fit(train_features[features_to_test], train_labels)
                p_pred = model.predict_proba(test_features[features_to_test])
                p_pred = p_pred[:,1]
                roc_value = roc_auc_score(test_labels, p_pred)

            if roc_value > best_roc:
                best_roc = roc_value
                current_best = feature 

        best_features.append(current_best)
        best_roc_list.append(best_roc)

        features.remove(current_best)
        print(f'Added Feature: {current_best} ROC: {best_roc}')
    return best_features, best_roc_list 

In [6]:
best_features, best_roc = ForwardSelection(X_train, y_train, X_test, y_test)

Added Feature: cms_tot_partd_payment_amt ROC: 0.6609055491005573
Added Feature: ccsp_239_ind ROC: 0.688086397436393
Added Feature: est_age ROC: 0.7099140499922565
Added Feature: cmsd2_men_mad_ind ROC: 0.7189037404378232
Added Feature: mabh_seg_H2 ROC: 0.7237759693676856
Added Feature: cons_n65p_y ROC: 0.7273631357194342
Added Feature: MobilityIndex ROC: 0.7308218968700965
Added Feature: cms_ra_factor_type_cd_CN ROC: 0.7343896413825268
Added Feature: betos_t1a_ind ROC: 0.7368300349681531
Added Feature: prov_spec_home_health_ind ROC: 0.7389353623502561
Added Feature: total_emer_visits ROC: 0.740577534130014
Added Feature: submcc_ben_othr_ind ROC: 0.7420777212112759
Added Feature: cons_homstat_Renter ROC: 0.7434794411462056
Added Feature: cons_hhcomp_Min Two People, Children ROC: 0.7446292771943014
Added Feature: betos_m5b_ind ROC: 0.7457012995642182
Added Feature: ccsp_220_ind ROC: 0.7470054365991584
Added Feature: reverse_raf ROC: 0.7479601068472752
Added Feature: betos_m1b_ind ROC: 0.7

ValueError: list.remove(x): x not in list

In [51]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42, n_jobs = -1)
print('model')
model = rf.fit(X_train, y_train)
print('predictions')
rf_predictions = model.predict(X_test)
print('roc')
roc_value = roc_auc_score(y_test, rf_predictions)
print('important')
important_features = model.feature_importances_
indices = np.argsort(important_features)[::-1]
pd.Series(important_features, index = indices).sort_values(ascending = True)

model
predictions
roc
important


311    0.000000e+00
148    0.000000e+00
97     0.000000e+00
212    0.000000e+00
269    0.000000e+00
337    0.000000e+00
39     0.000000e+00
5      0.000000e+00
305    0.000000e+00
14     0.000000e+00
127    0.000000e+00
571    0.000000e+00
240    0.000000e+00
9      0.000000e+00
56     0.000000e+00
594    0.000000e+00
12     0.000000e+00
25     0.000000e+00
349    0.000000e+00
49     0.000000e+00
421    0.000000e+00
465    0.000000e+00
327    0.000000e+00
8      1.341520e-07
331    1.692779e-07
171    4.810725e-07
515    8.470836e-07
199    1.131135e-06
355    1.525079e-06
3      1.849708e-06
600    1.862671e-06
213    2.208996e-06
473    3.424742e-06
416    3.636641e-06
404    3.723361e-06
194    4.420860e-06
310    4.678386e-06
365    4.934554e-06
256    6.411629e-06
522    7.269044e-06
593    1.013436e-05
135    1.028370e-05
13     1.042140e-05
41     1.415962e-05
380    1.417362e-05
234    1.449394e-05
119    1.449806e-05
456    1.637739e-05
414    1.663509e-05
364    1.793151e-05


In [52]:
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

In [53]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
feature_importances

,importance
cms_tot_partd_payment_amt,5.616861e-02
est_age,2.181106e-02
total_ambulance_visit_ct_pmpm,1.419864e-02
med_ambulance_visit_ct_pmpm,1.291792e-02
StressIndex,1.265310e-02
MobilityIndex,8.854886e-03
credit_bal_autobank,8.716926e-03
reverse_raf,8.546846e-03
credit_bal_1stmtg_30to59dpd,8.194243e-03
credit_hh_mtg_severederog,8.174616e-03


In [55]:
roc_value

0.7191156121776384

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

sffs = SFS(LogisticRegression(solver = 'liblinear'), 
          k_features=(1, 500), 
          forward=False, 
          floating=True,
          cv=0)
sffs.fit(X_train, y_train)
sffs.k_feature_names_